In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"status"

Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 


In [2]:
pkg"test"

   Testing Arborist
 Resolving package versions...
Test Summary: | Pass  Total
Arborist.jl   |    4      4
   Testing Arborist tests passed 


In [ ]:
Arborist.get_ast

In [20]:
using Arborist
using Arborist: graft, get_ast
using Test
using MacroTools

In [3]:
includet("../test/demo.jl")

In [6]:
badcake()

1

In [21]:
ast = get_ast(@which badcake1())

:(function badcake1()
      y = 0
      f = (()->begin
                  y += 1
              end)
      f()
      return y
  end)

In [40]:
dump(:(global x))

Expr
  head: Symbol global
  args: Array{Any}((1,))
    1: Symbol x


Test Summary:               | Pass  Total
is_generator_body_tier_pure |    8      8


Test.DefaultTestSet("is_generator_body_tier_pure", Any[DefaultTestSet("negatives", Any[], 4, false), DefaultTestSet("positives", Any[], 4, false)], 0, false)